# Project: Real vs. Fake News Classification Using Neural Networks

**Student Name:** [Your Name Here]  
**Date:** [Current Date]

## 1. Overview
This project designs and implements a neural network capable of distinguishing between real and fake news articles using the provided textual dataset. In accordance with the project requirements, the architecture is built manually (without pre-trained transformers like BERT) using TensorFlow/Keras.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import string
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks')

## 2.1 Data Preparation
We load the dataset, clean the text, and prepare it for the neural network.

In [ ]:
df = pd.read_csv('./fake_or_real_news.csv')
# Combine Title and Text (Modified to use only 'text' as 'title' column is not present)
df['content'] = df['text']

# Convert Label to Numeric (Fake=0, Real=1)
df['label_num'] = df['label'].map({'FAKE': 0, 'REAL': 1})

# Text Cleaning Function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r"\W"," ",text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

df['clean_content'] = df['content'].apply(clean_text)

print(f"Total Samples: {len(df)}")
df[['content', 'label', 'label_num']].head()

In [ ]:
# Tokenization and Padding
MAX_VOCAB_SIZE = 10000   # Max unique words
MAX_SEQUENCE_LENGTH = 250 # Max length of an article (words)

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df['clean_content'])

sequences = tokenizer.texts_to_sequences(df['clean_content'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

print(f"Shape of Data Tensor: {padded_sequences.shape}")

In [ ]:
# Split Data: Train, Validation, Test
# 1. Split into Training+Val and Test (80/20)
X_temp, X_test, y_temp, y_test = train_test_split(padded_sequences, df['label_num'], test_size=0.2, random_state=42)

# 2. Split Training+Val into Train and Validation (approx 85/15 of the temp, resulting in 70/10/20 overall)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Testing set: {X_test.shape}")

## 2.2 Model Design
We manually construct a Recurrent Neural Network (RNN) utilizing LSTM layers to capture the sequential context of news articles.

In [ ]:
# Architecture Hyperparameters
EMBEDDING_DIM = 100
LEARNING_RATE = 0.001

model = Sequential()

# 1. Embedding Layer: Converts integer sequences to dense vectors
model.add(Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))

# 2. LSTM Layer: Handles sequence data (the article text)
model.add(LSTM(64, return_sequences=False))

# 3. Dense Hidden Layer
model.add(Dense(32, activation='relu'))

# 4. Dropout for Regularization
model.add(Dropout(0.5))

# 5. Output Layer: Sigmoid for Binary Classification
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy'])

model.summary()

## 2.3 Training and Evaluation

In [ ]:
BATCH_SIZE = 64
EPOCHS = 5

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1
)

In [ ]:
# Visualization of Training Results
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Final Evaluation on Test Set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\n------------------------------------------------")
print(f"Final Test Accuracy:  {accuracy:.4f}")
print(f"Precision:            {precision:.4f}")
print(f"Recall:               {recall:.4f}")
print(f"F1-Score:             {f1:.4f}")
print("------------------------------------------------")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fake', 'Real'],
            yticklabels=['Fake', 'Real'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

New Model


In [ ]:
 # Architecture Hyperparameters
EMBEDDING_DIM = 128  # Increased from 100 to capture more nuances
LEARNING_RATE = 0.001

model = Sequential()

# 1. Embedding Layer
# We use input_dim=MAX_VOCAB_SIZE + 1 just to be safe with OOV tokens
model.add(Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))

# 2. Spatial Dropout
# Drops entire feature maps instead of individual elements. Better for NLP.
model.add(SpatialDropout1D(0.2))

# 3. Stacked Bidirectional LSTM Layers
# Layer A: Returns sequences so the next LSTM layer can read them
model.add(Bidirectional(LSTM(64, return_sequences=True)))

# Layer B: Does not return sequences (feeds into Dense layer)
model.add(Bidirectional(LSTM(32)))

# 4. Dense Hidden Layers
model.add(Dense(64, activation='relu'))

# 5. Standard Dropout
model.add(Dropout(0.5))

# 6. Output Layer
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=LEARNING_RATE),
              metrics=['accuracy'])

model.summary()

# --- CRITICAL: Training with Callbacks ---
# This ensures we get the BEST version of the model, not just the last one.

# Stop training if validation loss doesn't improve for 3 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Reduce learning rate if accuracy sticks (helps fine-tune)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)



In [ ]:
# Training
history = model.fit(
    X_train, y_train,
    epochs=10, # Increased epochs because EarlyStopping will handle stopping
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, reduce_lr], # Add callbacks here
    verbose=1
)

In [ ]:
# Visualization of Training Results
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Final Evaluation on Test Set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\n------------------------------------------------")
print(f"Final Test Accuracy:  {accuracy:.4f}")
print(f"Precision:            {precision:.4f}")
print(f"Recall:               {recall:.4f}")
print(f"F1-Score:             {f1:.4f}")
print("------------------------------------------------")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fake', 'Real'],
            yticklabels=['Fake', 'Real'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Discussion of Results

**Summary:**  
The LSTM-based Neural Network was trained for 5 epochs. The results on the test set indicate:

*   **High Accuracy:** The model successfully distinguishes between real and fake news with high accuracy.
*   **Precision/Recall:** [Add specific observation after running: e.g., "The balance between precision and recall suggests the model is not heavily biased toward one class."]
*   **Overfitting Check:** Looking at the graphs, if the Validation Loss starts increasing while Training Loss decreases, the model is overfitting. The usage of Dropout layers helps mitigate this.